# Augmenter l'accuracy du Computer Vision grâce aux réseaux à convolution

Dans le workshop précédent, vous avez vu comment reconnaître des vêtements à travers un réseau de neurones constitué de 3 couches. Vous avez pu experimenté l'impact des différents paramètres du modèle comme le nombre de neurones dans la couche cachée, le nombre d'epochs, etc. sur l'accuracy final.

Nous vous avons remis le code précedents pour un petit rappel. Exécutez la cellule suivante et retenez l'accuracy affichée à la fin de l'entraînement.

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images / 255.0
test_images=test_images / 255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5)

test_loss = model.evaluate(test_images, test_labels)

Votre accuracy est probablement égal à 89% sur le jeu de train et 87% sur le jeu de validation... pas trop mal... Mais comment peut-on améliorer ce score ? Nous pouvons utiliser les convolutions. Nous ne donnerons pas trop de détails ici, mais le concept des réseaux de neurones à convolution est d'apprendre à detecter des patterns spécifiques sur le contenu d'une image.

Si vous avez déjà fait du processing d'images en utilisant un filtre (comme ici: https://en.wikipedia.org/wiki/Kernel_(image_processing), alors vous serez très familiers avec les convolutions.

En bref, vous prenez un filtre (généralement 3x3 ou 5x5) et vous le passez sur l'image. En modifiant les pixels sous-jacents en fonction de la formule de ce filtre représenté par une matrice, vous pouvez effectuer des opérations telles que la détection des contours. Ainsi, par exemple, si vous examinez le lien ci-dessus, vous verrez que pour un filtre de 3x3 défini pour la détection de contours, la cellule du milieu est définie à 8 et tous ses voisins à -1. Dans ce cas, pour chaque pixel, vous allez multiplier sa valeur par 8, puis y soustraire la valeur de chaque voisin. En le faisant pour chaque pixel, vous obtiendrez une nouvelle image dont les contours sont améliorés.

C'est parfait pour le computer vision, parce que souvent les features qui définissent un objet ne représentent qu'une partie de l'image entière, et que l'information dont on a besoin est beaucoup plus faible que tous les pixels de l'image. Ce concept nous permet alors de nous focaliser uniquement sur des features qui sont mises en valeur. 

En ajoutant des couches de convolution avant vos couches dense, l'information fournie aux dense layers est bien plus ciblée, et potentiellement plus précise.

Exécutez le code suivant: il s'agit du même code précedent, mais cette fois avec des couches de convolution ajoutées au début. Cela prendra un peu plus de temps, mais regardez l'impact qu'il aura sur l'accuracy:

In [ ]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(training_images, training_labels, epochs=5)
test_loss = model.evaluate(test_images, test_labels)


Vous devriez obtenir quelque chose comme 93% sur les données de train et 91% sur les données de validation. Cela est significatif, vous êtes sur la bonne direction !

Essayez d'exécuter le code sur plus d'epochs, comme 20 par exemple, et regardez les résultats ! Mais malgré que les résultats du train deviennent de mieux en mieux, les résultats de la validation ont tendance à diminuer, dû à l'overfitting.

Revisualisez le code ci-dessus, puis regardez étape par étape comment les convolutions sont construites:

La première étape est de récupérer les données. Vous verrez qu'il y a quelques changements car les données de train doivent être reshaped. En effet, la première convolution attend un tensor simple contenant toutes les données. Donc au lieu de donner 60000 images de 28x28x1 dans une liste, nous devons donner un seul array 4D de 60000x28x28x1, et ceci de même pour les images de test. Si vous ne faîtes pas cela, vous aurez une erreur pendant l'entraînement car les couches de convolutions n'aurait pas réussi à reconnaître les shapes.

```
import tensorflow as tf
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
```

L'étape suivante est de définir votre modèle. Maintenant au lieu de donner en premier votre couche d'input, vous devez ajouter d'abord une couche de convolution. Les paramètres sont les suivants:
1. Le nombre de convolutions que vous voulez generer. Ce nombre est complètement arbitraire mais il faut généralement commencer avec un nombre multiple de 32.
2. La size de la convolution, dans notre cas une grille de 3x3
3. La fonction d'activation, dans notre cas la ReLU qui rappelons, équivaut à x quand x > 0 sinon 0
4. Dans la première couche, le shape des données en entrée.

Vous suivrez ensuite avec une couche de MaxPooling qui sert à compresser l'image, en gardant les aspects les plus importants déterminés par la convolution. En spécifiant (2,2) pour le MaxPooling, l'effet sera de diviser par 4 la taille de l'image. Sans aller trop loin dans les détails, l'idée est qu'il crée un array de pixels 2x2,récupère la plus grande valeur de cette array, et transforme ces 4 pixels en 1 pixel. De façon itérative à travers tout l'image, le nombre de pixels à l'horizontal et à la verticale sera divisé chacun par 2, ce qui réduira l'image de 25%.

Vous pouvez appeler la méthode model.summary() pour visualiser les size et shape de votre réseau, et vous verrez qu'après chaque couche de MaxPooling, la size de l'image est divisée.

```
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
```

Puis ajoutez une autre convolution



```
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2)
```



Et maintenant flatten l'output. Après cela vous aurez exactement la même structure de réseau de neurones que la version non convolution 

```
  tf.keras.layers.Flatten(),
```



Les mêmes couches dense 128, et 10 pour l'output que dans l'exemple de pré-convolution.


```
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
```



Maintenant vous pouvez compiler votre modèle, faire appel à la méthode fit pour l'entraînement, et évaluer la loss et l'accuracy grâce au jeu de validation.


```
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)
```


# Visualisation des convolutions et du pooling

Ce code va vous permetter de visualiser les convolutions graphiquement. Le `print(test_labels[;100])` vous montre les 100 premiers labels réels du jeu de test, et vous pouvez voir que ceux des inder 0, 23 et 28 ont la même valeur (9). Il s'agit tous de chaussures. Jetez un oeil sur le résultat de la convolution sur chacun de ces index, et vous commencerez à voir quelques features qu'ils ont en commun sortir. N'hésitez pas à changer les valeurs des index pour visualiser différentes images.

In [ ]:
print(test_labels[:100])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
f, axarr = plt.subplots(3,4)
FIRST_IMAGE=0
SECOND_IMAGE=23
THIRD_IMAGE=28
CONVOLUTION_NUMBER = 1
from tensorflow.keras import models
layer_outputs = [layer.output for layer in model.layers]
activation_model = tf.keras.models.Model(inputs = model.input, outputs = layer_outputs)
for x in range(0,4):
  f1 = activation_model.predict(test_images[FIRST_IMAGE].reshape(1, 28, 28, 1))[x]
  axarr[0,x].imshow(f1[0, : , :, CONVOLUTION_NUMBER], cmap='inferno')
  axarr[0,x].grid(False)
  f2 = activation_model.predict(test_images[SECOND_IMAGE].reshape(1, 28, 28, 1))[x]
  axarr[1,x].imshow(f2[0, : , :, CONVOLUTION_NUMBER], cmap='inferno')
  axarr[1,x].grid(False)
  f3 = activation_model.predict(test_images[THIRD_IMAGE].reshape(1, 28, 28, 1))[x]
  axarr[2,x].imshow(f3[0, : , :, CONVOLUTION_NUMBER], cmap='inferno')
  axarr[2,x].grid(False)

**Exercices:**

1. Essayer de modifier les convolutions. Changez les 32s en 16 ou 64. Quel impact cela aura sur l'accuracy et/ou le temps d'entraînement ?

2. Retirez la convolution finale. Quel impact cela aura sur l'accuracy ou le temps d'entraînement ?

3. Pourquoi ne pas ajouter plus de convolution ? Quel impact cela aura sur l'accuracy ou le temps d'entraînement ? Expérimentez.

4. Retirer toutes les convolutions sauf la première ? Quel sera l'impact ? Expérimentez.

5. Dans le workshop précent vous avez implémenté un callback pour vérifier votre fonction de loss et stopper l'entraînement dès qu'une certaine valeur est atteinte. Essayez de voir si vous pouvez l'implémenter ici !

In [ ]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)